# 2D Anisotropic Diffusion (non coercive case)

We consider the study and analysis of the eigenvalue problem associated to the following anisotropic diffusion operator
$$
    - \nabla \cdot \left( \mathbf{b} \otimes \mathbf{b} \nabla \phi \right) = \omega^2 \phi
$$
where $\mathbf{b} = (\iota, 1)$ is the normalized vector of the magnetic field.

In fact, the original problem is 3D and the iota profile is given as

$$
\iota(r) = 0.47262 + 0.32392 r + 0.49604 r^2 + 5.3991 . 10^{-6} r^3 - 3.165 . 10^{-5} r^4 + 4.7963 . 10^{-5} r^5 - 2.2824 . 10^{-5} r^6
$$

which is the LHD-$\iota$-profile. 

## Formal model

In [19]:
from sympy.core.containers import Tuple
from sympy import Function
from IPython.display import Math

from sympde.core     import Constant
from sympde.calculus import grad, dot
from sympde.topology import ScalarFunctionSpace
from sympde.topology import Domain
from sympde.topology import elements_of
from sympde.expr     import BilinearForm
from sympde.expr     import integral

DIM = 2
domain = Domain('\Omega', dim=DIM)
x,y = domain.coordinates

V = ScalarFunctionSpace('V', domain)

u,v = elements_of(V, names='u,v')

#bx = Constant('b_x')
#by = Constant('b_y')
#b  = Tuple(bx, by)

bx = Function('b_x')
by = Function('b_y')
b = Tuple(bx(x,y), by(x,y))

expr = integral(domain, dot(b, grad(v)) * dot(b, grad(u))) 
a = BilinearForm((u,v), expr)

mass = BilinearForm((u,v), integral(domain, u*v))

## Symbolic expression

In [20]:
# imports from gelato
from gelato.expr     import gelatize
from gelato.printing import latex

We compute the GLT expression by calling the function **gelatize** for a given degree $p$

In [21]:
glt = gelatize(a, degrees=[3,3])

In [22]:
Math(latex(glt))

<IPython.core.display.Math object>

In [23]:
# imports from sympy
from sympy import Symbol
from sympy import ratsimp
from sympy import expand

tx = Symbol("tx")
ty = Symbol("ty")
nx = Symbol("nx", integer=True)
ny = Symbol("ny", integer=True)
N  = Symbol("N", integer=True)
wx = Symbol('\omega_x')
wy = Symbol('\omega_y')

In [24]:
p = 3
degrees = [p,p]

sa = gelatize(a, degrees=degrees)
sm = gelatize(mass, degrees=degrees)

expr = sa/sm

In [25]:
expr = expr.subs({nx: N, ny: N})

In [26]:
Math(latex(expr))

<IPython.core.display.Math object>

In [27]:
Math(latex(ratsimp(expr/N**2)))

<IPython.core.display.Math object>

In [28]:
expr = expr.subs({tx: wx / N, ty: wy / N})
expr = expr.subs({nx: N-p, ny: N-p}) #N = nx + p
expr = expand(expr)

In [32]:
e_exact = (wx*bx(x,y) + wy*by(x,y))**2

# here we consider an expansion up to the forth order
order = 8
e = expr - e_exact.expand()
e = e.series(1/N, 0, order)
Math(latex(e))

<IPython.core.display.Math object>

In [4]:
# css style
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()